Этот скрипт заходит на ютуб-каналы, переходит по ссылкам на видео, парсит разную нужную информацию, а потом записывает её в файл pickle

In [1]:
import json
import pickle
import requests
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

Создадим класс «страница»

In [2]:
class Page:
    def __init__(self, url):
        self.url = url
        self.html = ''
    
    @staticmethod   
    def scroll_down(driver, max_iter):
        pass
    
    @staticmethod
    def button_to_videos(self):
        pass
    
    def get_source(self):
        driver = create_driver(headless=False)
        driver.get(self.url)
    #   driver.refresh()  # помогает если ютуб просит капчу
        time.sleep(3.5)
        self.button_to_videos(driver)
        self.scroll_down(driver, max_iter=3) #вписал 3 три для скорости
        html = driver.page_source
        driver.close()
        self.html = html

Дочерний класс «страница канала»

In [3]:
class ChannelPage(Page):
    
    @staticmethod
    def button_to_videos(driver):
        buttons_selector = '#tabsContent > tp-yt-paper-tab:nth-child(4)'
        try:
            video_button = driver.find_element_by_css_selector(buttons_selector)
            video_button.click()
        except:          
            video_button = driver.find_element_by_css_selector('#accept-button')
            video_button.click()
            video_button = driver.find_element_by_css_selector(buttons_selector)
            video_button.click()
    
    @staticmethod
    def scroll_down(driver, max_iter=15):
        counter = 1
        flag = True
        while (flag and counter < max_iter):
            counter += 1
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            dates = soup.find_all(id='metadata-line')
            for date in dates:
                date=date.text
                if '2 года назад' in date or '3 года назад' in date or '4 года назад' in date:
                    flag = False
                    print(date)
                    print('Долистал до старых видео')
                    break
            html = driver.find_element_by_tag_name('html')
            html.send_keys(Keys.END)
            time.sleep(2)
            dates = []
        dates = []
        html = ''
        
    def get_videos(self, name):
        videos_for_export = []
        soup = BeautifulSoup(self.html, 'html.parser')
        videos = soup.find_all(id='video-title')
        print(name + '  — ' + str(len(videos)) + ' видео')
        for current_url in videos:
            try:
                current = VideoPage(current_url)
                current.get_source()
            except Exception as error:
                print('Ошибка получения видео:', error)
            try:
                video = Video(current.html, name)
                videos_for_export.append(video)
            except Exception as error:
                print('Ошибка сохранения видео:', error)
            current = ''
        return videos_for_export

И дочерний класс «страница видео»

In [4]:
class VideoPage(Page):   
    def __init__(self, url):
        url = url['href']
        url = 'https://www.youtube.com' + url
        return Page.__init__(self, url)

Теперь импортируем классы для хранения скачанной информации

In [5]:
from classes_for_project import Channel
from classes_for_project import Video

In [6]:
def create_driver(headless=False):
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    #options.set_headless(headless=headless) #эта строчка прячет|открывает окно, которое открывает selenium4
    options.headless=headless #эта строчка как предыдущая, но не появляется варнинг
    driver = webdriver.Chrome(options=options)
    return driver

Пройдёмся по словарю со ссылками на каналы и для каждой ссылки на канал спарсим его

In [7]:
links = { 'other' : ['https://www.youtube.com/user/hopehill2000/featured'],
    'math' : ['https://www.youtube.com/channel/UCASlwNxf7mHBUEPr1s6fsDg']}

In [8]:
clear_log = True

In [9]:
if clear_log:
    log_name = 'project_log.txt'
    with open(log_name, 'w') as f:
        f.write('')

In [10]:
%%time
for category in links: 
    for url in links[category]:
        f = open(log_name, 'r')
        logs = f.read()
        logs = logs.split('sep\n')
        f.close()
        if url not in logs:      
            try:
                current_page = ChannelPage(url)
                current_page.get_source()
                try:
                    current_channel = Channel(current_page.html, category, url)
                    if current_channel.name not in logs:
                        videos = current_page.get_videos(current_channel.name)
                        current_channel.videos = videos
                        current_channel.pirkovskii_here()
                        videos = []
                        print('канал ', current_channel.name, 'скачан')
                        current_channel.dump()
                        f = open(log_name, 'r')
                        current_file = f.read()
                        f.close()
                        f = open(log_name, 'w')
                        try:
                            f.write(current_file + 'sep\n' + url + 'sep\n' + current_channel.name)
                        except:
                             f.write(current_file + 'sep\n' + url + 'sep\n' + 'bad_name')
                        f.close()
                        print('канал ', current_channel.name, 'сохранён')
                    else:
                        try:
                            f.write(current_file + 'sep\n' + url + 'sep\n' + current_channel.name)
                        except:
                             f.write(current_file + 'sep\n' + url + 'sep\n' + 'bad_name')
                        f.close()
                        print('канал ', current_channel.name, 'уже сохранён')
                except Exception as error:
                    print('Ошибка:', error)
            except Exception as error:
                print('Ошибка get_source:', error)
        else:
            print('канал ', url.split('/')[-1], 'уже сохранён')
    try:
        print('категория ', category, 'сохранена')
    except Exception as error:
        print('Ошибка cохранения категории:', error)

hopehill2000  — 3 видео
канал  hopehill2000 скачан
канал  hopehill2000 сохранён
категория  other сохранена
Mathematics at HSE  — 90 видео
канал  Mathematics at HSE скачан
канал  Mathematics at HSE сохранён
категория  math сохранена
Wall time: 28min 8s
